## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
init_notebook_mode(True)

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo

## Day 1: Cleaning, Plots

In [ ]:
plotdir = hpc_ext_path+'plots/magnet_stability/2019-07-16/'

In [ ]:
pkldir = hpc_ext_path+"datafiles/magnet_stability/pkl/"
directory = "/home/ckampa/Dropbox/LogFiles/"#hpc_ext_path+'datafiles/magnet_stability/nmr/'
filename = '2019-07-16 101205slow.txt'

In [ ]:
# read pickled files!
df = pd.read_pickle(pkldir+"run10.pkl")
df_full = pd.read_pickle(pkldir+"run10_full.pkl")

In [ ]:
df = pd.read_csv(directory+filename)

In [ ]:
df_full = pd.read_csv(directory+filename)

In [ ]:
df.describe()

In [ ]:
df = df[df[" NMR [T]"] != 0]

In [ ]:
fig = histo(df[' NMR [T]'],bins=50,yscale='linear',inline=True);

In [ ]:
fname = 'hist_NMR'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html',auto_open=False)

In [ ]:
len(df_full)

In [ ]:
dates = [parser.parse(row.Time) for row in df.itertuples()]# if isdatetime(row.Time.values[0])]

- NOTE: 'as.type("str")' is necessary to ensure plotly doesn't convert datetime to UTC!
    - **Should not be necessary in updated Plotly package**

In [ ]:
df["Datetime"] = pd.to_datetime(dates)#.astype("str")
df = df.set_index("Datetime")

In [ ]:
dates = [parser.parse(row.Time) for row in df_full.itertuples()]# if isdatetime(row.Time.values[0])]

- NOTE: 'as.type("str")' is necessary to ensure plotly doesn't convert datetime to UTC!

In [ ]:
df_full["Datetime"] = pd.to_datetime(dates)#.astype("str")
df_full = df_full.set_index("Datetime")

In [ ]:
column = " NMR [T]"
fig1 = scatter2d(df.index,df[column],show_plot=False,inline=True)
fig2 = histo(df[column],bins=50, autobin=False, horizontal=True,show_plot=False, inline=True)
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

In [ ]:
fig.layout.title = "NMR [T] vs. Datetime"
fig.layout.xaxis.title = "Datetime"
fig.layout.yaxis.title = "NMR [T]"

In [ ]:
fname = 'scatter_hist_NMR'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html',auto_open=False)

In [ ]:
fig = scatter2d(df_full.index,df_full[" NMR [T]"],lines=True,markers=False,size_list=[6],inline=True)

In [ ]:
pio.write_image(fig, plotdir+'NMR_vs_Time_full.pdf')
pio.write_image(fig, plotdir+'NMR_vs_Time_full.png')
plot(fig, filename=plotdir+'NMR_vs_Time_full.html', auto_open=False)

In [ ]:
fig = scatter2d(df.index,df[" NMR [T]"],lines=True,markers=False,size_list=[6],inline=True)

In [ ]:
nmr =df[' NMR [T]']
(nmr.max() - nmr.min()) / nmr.mean()

In [ ]:
pio.write_image(fig, plotdir+'NMR_vs_Time.pdf')
pio.write_image(fig, plotdir+'NMR_vs_Time.png')
plot(fig, filename=plotdir+'NMR_vs_Time.html', auto_open=False)

In [ ]:
df.columns

In [ ]:
#df.columns ## OLD

In [ ]:
len(df.columns)

In [ ]:
#df = df_full

In [ ]:
temps_ch = [a for a in df.columns if ("CH" in a) and int(a[-2:])<21]# or "HVAC" in a]
temps_ch.append(" Hall Element")
temps_ch.append(" Floor")
temps_ch.append(" Roof")
temps_ch.append(" Yoke")
temps_ch.append("Yoke (center magnet)")

In [ ]:
temps_ch

In [ ]:
df[temps_ch].describe()

In [ ]:
fig = plt.figure()
for c in temps_ch:
    df[c].plot()
#plt.ylabel('Temp [C]')
plt.ylabel("Temp [$^{\circ}$ C]")
plt.legend()    

In [ ]:
fig = scatter2d([df.index for c in temps_ch],[df[c] for c in temps_ch],inline=True);
fig.layout.title = "Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"

In [ ]:
#fname = 'temps_scatter'
fname = 'temps_scatter_full'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
fig = scatter2d(df.index,df[" Chamber Wall CH20"],inline=True)

In [ ]:
fig = scatter2d([df.index for c in temps_ch],[df[c]-(df[c].mean()-df[" Floor"].mean()) for c in temps_ch],inline=True);
fig.layout.title = "Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"

In [ ]:
fname = 'temps_scatter_adjusted'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html',auto_open=False)

In [ ]:
fig = histo([df[c]-(df[c].mean()-df[" Floor"].mean()) for c in temps_ch],xlabel="Temperature (deg C)",verbosity=1,opacity=0.6,inline=True);
fig.layout.title = "Temp Histos"

In [ ]:
fname = 'hist_temps_adjusted'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html',auto_open=False)

In [ ]:
fig = histo([df[c] for c in temps_ch],xlabel="Temperature (deg C)",verbosity=1,inline=True);
fig.layout.title = "Temp Histos"

In [ ]:
fname = 'hist_temps'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html')

In [ ]:
#fig = scatter2d(df.index,df[" Yoke"],inline=True);
fig = histo(-df[" Yoke"],xlabel="-1 * Temperature (deg C)", bins=50,autobin=False,inline=True);

In [ ]:
fname = 'hist_Yoke'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html')

In [ ]:
df.columns

In [ ]:
fig = scatter2d(2*[df.index],[df["Desk Ambient"], df[" Spare Ambient on desk"]],inline=True);

In [ ]:
pio.write_image(fig,plotdir+"Desk_comparison.pdf")
pio.write_image(fig,plotdir+"Desk_comparison.png")

In [ ]:
fig = scatter2d([df.index],[df["Desk Ambient"]-df[" Spare Ambient on desk"]],inline=True);

In [ ]:
fig.layout.title = "Desk Ambient Residuals vs. Time"
fig.layout.yaxis.title = "Desk Ambient - Spare Ambient on desk (deg C)"
fig.layout.showlegend = False

In [ ]:
pio.write_image(fig,plotdir+"Desk_comparison_subtraction.pdf")
pio.write_image(fig,plotdir+"Desk_comparison_subtraction.png")

In [ ]:
div = 1000
fig = histo(pd.Series(df["Desk Ambient"][:div]-df[" Spare Ambient on desk"][:div], name="Residuals"), bins=50, autobin=False, inline=True)
fig = histo(pd.Series(df["Desk Ambient"][div:]-df[" Spare Ambient on desk"][div:], name="Residuals"), bins=50, autobin=False, inline=True)

In [ ]:
fig.layout.title = "Desk Ambient Residuals"
fig.layout.xaxis.title = "Desk Ambient - Spare Ambient on desk (deg C)"
#fig.layout.showlegend = False

In [ ]:
pio.write_image(fig,plotdir+"hist_Desk_comparison_subtraction.pdf")
pio.write_image(fig,plotdir+"hist_Desk_comparison_subtraction.png")

In [ ]:
fig = histo(df["Desk Ambient"],bins=40,autobin=False,inline=True);
pio.write_image(fig,plotdir+"hist_temp_Desk.pdf")
pio.write_image(fig,plotdir+"hist_temp_Desk.png")
fig = histo(df[" Floor"],bins=40,autobin=False,inline=True);
pio.write_image(fig,plotdir+"hist_temp_Floor.pdf")
pio.write_image(fig,plotdir+"hist_temp_Floor.png")

In [ ]:
fig = histo(df[" Roof"],bins=50,autobin=False,inline=True);

In [ ]:
df[' Magnet Current [A]'].plot()

In [ ]:
fig = scatter2d(df.index,df[' Magnet Current [A]'],inline=True);

In [ ]:
fname = 'magnet_current'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
fig = scatter2d(df.index,df[' Magnet Voltage [V]'],inline=True);

In [ ]:
fname = 'magnet_voltage'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
m =df[' Magnet Voltage [V]']
(m.max() - m.min()) / m.mean()

In [ ]:
for col in df.columns:
    fig = scatter2d(df.index,df[col],lines=True,markers=False,size_list=[6], show_plot=False,inline=True);
    pio.write_image(fig,plotdir+f"time_series/{col}.pdf")
    pio.write_image(fig,plotdir+f"time_series/{col}.png")

In [ ]:
for col in df_full.columns:
    fig = scatter2d(df_full.index,df_full[col],lines=True,markers=False,size_list=[6], show_plot=False,inline=True);
    pio.write_image(fig,plotdir+f"time_series_full/{col}.pdf")
    pio.write_image(fig,plotdir+f"time_series_full/{col}.png")

## Correlations

- As seen in past dataset, NMR reading seems inversely correlated to the magnet voltage.
- It also appeared that the voltage was correlated to the water cooling system.
- Let's check if this relationship holds and also try to tease out other correlations.

In [ ]:
import operator
from scipy import stats

In [ ]:
ns = []
for col in df.columns:
    if (df[col].dtype == np.float64) & ("Zab" not in col) & ("NMR [FFT]" not in col) & (col != " "):
        ns.append(col)

In [ ]:
len(df.columns)

In [ ]:
len(ns)

In [ ]:
df[ns].corr()

In [ ]:
plt.matshow(df[ns].corr())

In [ ]:
f = plt.figure(figsize=(25, 15))
plt.matshow(df[ns].corr(), fignum=f.number)
plt.xticks(range(df[ns].shape[1]), df[ns].columns, fontsize=14, rotation=55)
plt.yticks(range(df[ns].shape[1]), df[ns].columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
f = plt.figure(figsize=(25, 15))
plt.matshow(df[temps_ch].corr(), fignum=f.number)
plt.xticks(range(df[temps_ch].shape[1]), df[temps_ch].columns, fontsize=14, rotation=55)
plt.yticks(range(df[temps_ch].shape[1]), df[temps_ch].columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
#corr.style.background_gradient(cmap='coolwarm', axis=None)
#df.corr().style.background_gradient(cmap='coolwarm', axis=None)
df[ns].corr().style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
primary_corr = dict()
for col in df.columns:
    if df[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df[col], df[" NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

sorted_primary_corr

Would be helpful to see time as the color to see how this is traced out.

In [ ]:
df['ind'] = np.array([i for i in range(len(df))])

In [ ]:
def color_scatter2d(x,y,ind,time=df.index,show_plot=True):
    data = [
        go.Scatter(
            x=x,
            y=y,
            marker=dict(
                size=2,
                cmax=ind.max(),
                cmin=ind.min(),
                color=ind,
                colorbar=dict(
                    title='time (index)',
                    #ticktext=df.index
                ),
                colorscale='Viridis',
            ),
            mode='markers',
            text=time,
        )
    ]
    
    layout = go.Layout(
        title = f"{y.name} vs. {x.name}",
        xaxis = dict(
            title = f"{x.name}"
        ),
        yaxis = dict(
            title = f"{y.name}"
        ),
    )

    fig = go.Figure(data=data,layout=layout)
    if show_plot:
        iplot(fig)
    return fig

In [ ]:
for col in df.columns:
    fig = color_scatter2d(df[col],df[" NMR [T]"],df.ind,show_plot=False);
    pio.write_image(fig,plotdir+f"nmr_correlations/{col}_vs_nmr.pdf")
    pio.write_image(fig,plotdir+f"nmr_correlations/{col}_vs_nmr.png")

In [ ]:
df_back = df

In [ ]:
df = df_back

In [ ]:
df_ = df[df.ind < 1600]

In [ ]:
fig = color_scatter2d(df[" Yoke"],df[" NMR [T]"],df.ind,show_plot=True);

In [ ]:
df = df_

In [ ]:
#corr_main = " LCW in barrel"
corr_main = " Yoke"
#corr_main = " Floor"

filestart = corr_main.strip().lower()

#slope, intercept, r_value, p_value, std_err = stats.linregress(df[" Yoke"], df[" NMR [T]"])
slope, intercept, r_value, p_value, std_err = stats.linregress(df[corr_main], df[" NMR [T]"])

In [ ]:
slope, intercept, r_value, p_value, std_err

In [ ]:
line = slope * df[corr_main] + intercept

In [ ]:
fig = scatter2d(2*[df[corr_main]],[df[" NMR [T]"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit.png")

In [ ]:
df["NMR_const"] = df[" NMR [T]"] - slope * df[corr_main] - intercept + df[" NMR [T]"].mean()

In [ ]:
column = "NMR_const"
fig1 = scatter2d(df.index,df[column],show_plot=False,inline=True)
fig2 = histo(df[column],bins=50, autobin=False, horizontal=True,show_plot=False, inline=True)
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)

In [ ]:
fig.layout.title = "NMR_const [T] vs. Datetime"
fig.layout.xaxis.title = "Datetime"
fig.layout.yaxis.title = "NMR_const [T]"

In [ ]:
fname = 'scatter_hist_NMR_const'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html',auto_open=False)

In [ ]:
column = "NMR_const"
fig1 = scatter2d(df.index,df[column],show_plot=False,inline=True)
fig2 = histo(df[column],bins=50, autobin=False, horizontal=True,show_plot=False, inline=True)
fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], shared_yaxes=True)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.layout.yaxis.title = "Corrected NMR (T)"
iplot(fig)

In [ ]:
fig = scatter2d(2*[df[corr_main]],[df["NMR_const"],line],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.png")

In [ ]:
fig = histo(df[" NMR [T]"],bins=50,autobin=False,inline=True);
pio.write_image(fig,plotdir+f"hist_NMR.pdf")
pio.write_image(fig,plotdir+f"hist_NMR.png")
fig = histo(df["NMR_const"],bins=50,autobin=False,inline=True);
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.pdf")
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.png")

This looks nice. Let's try to remake the correlation plots on NMR_const.

In [ ]:
secondary_corr = dict()
for col in df.columns:
    if df[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df[col], df["NMR_const"])
        #print(col,r_value_v,p_value_v,std_err_v)
        secondary_corr[col] = r_value_v

sorted_corr = sorted(secondary_corr.items(), key=operator.itemgetter(1))

sorted_corr

In [ ]:
for col in df.columns:
    fig = color_scatter2d(df[col],df["NMR_const"],df.ind,show_plot=False);
    pio.write_image(fig,plotdir+f"nmr_const_correlations/{col}_vs_nmr.pdf")
    pio.write_image(fig,plotdir+f"nmr_const_correlations/{col}_vs_nmr.png")

The water temperature or maybe magnet voltage seem correlated now, except for a weird bit at the end. So we do a similar fit and subtract based on data with a time cut.

In [ ]:
#fit_var = "Coil 1 (new)"
#fit_var = " ICW He2 out"
#fit_var = " CH15"
#fit_var = " Magnet Current [V]"
#fit_var = " LCW from PS"
fit_var = " PS internal"
fig = color_scatter2d(df[fit_var],df["NMR_const"],df.ind,show_plot=True);

In [ ]:
#fit_var = " Magnet Voltage [V]"
#t = df.ind < 3400
#t = df.ind > 1500
t = df.ind >= 0
#fig = color_scatter2d(df[t][fit_var],df[t]["NMR_const"],df.ind,show_plot=True);

In [ ]:
slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df[t][fit_var], df[t]["NMR_const"])

In [ ]:
slope_v, intercept_v, r_value_v, p_value_v, std_err_v

In [ ]:
line_v = slope_v * df[fit_var] + intercept_v

In [ ]:
fig = scatter2d(2*[df[fit_var]],[df["NMR_const"],line_v],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
df["NMR_const_v"] = df["NMR_const"] - slope_v * df[fit_var] - intercept_v + df["NMR_const"].mean()

In [ ]:
#fig = scatter2d(2*[df[fit_var]],[df["NMR_const_v"],line_v],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
#fig = scatter2d(2*[df[" Yoke"]],[df["NMR_const"],df["NMR_const_v"]],lines=False,markers=True,size_list=[2,2],inline=True)

In [ ]:
histo(df[" NMR [T]"],bins=50,autobin=False,inline=True);
histo(df["NMR_const"],bins=50,autobin=False,inline=True);
histo(df["NMR_const_v"],bins=50,autobin=False,inline=True);

## Slope of changes
- How quickly do the correlated temperatures change?

In [ ]:
df.drop([" "],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
col_float = [col for col in df.columns if df[col].dtype == np.float64]

In [ ]:
col_float

In [ ]:
len(col_float)

In [ ]:
#temp = df[' Yoke']

In [ ]:
slopes = df[col_float].apply(lambda x: np.polyfit(df.ind, x, 1)[0])

In [ ]:
slopes

In [ ]:
[df[" NMR [T]"][i+1] - row[" NMR [T]"] for i,row in enumerate(df.itertuples())]

In [ ]:
# smooth temperature

In [ ]:
df["Hall_smooth"] = df[" Hall Element"].ewm(span = 10).mean()

In [ ]:
t_outside = df["Desk Ambient"].values
t_inside2 = df[" Hall Element"].values
t_inside = df[" Yoke"].values
#t_inside2 = df[" Floor"].values

In [ ]:
slope_inside = [t_inside[i+1]-t_inside[i] for i in range(len(t_inside)-1)]
slope_inside_coarse = [(t_inside[i+5]-t_inside[i])/5 for i in range(len(t_inside)-5)]
slope_inside_coarse = pd.Series(slope_inside,name="Inside Temp Change Coarse: Hall Element")
slope_inside.append(0)
slope_inside = pd.Series(slope_inside,name="Inside Temp Change: Yoke")
slope_inside2 = [t_inside[i+1]-t_inside[i] for i in range(len(t_inside)-1)]
slope_inside2.append(0)
slope_inside2 = pd.Series(slope_inside2,name="Inside Temp Change: Hall Element")

In [ ]:
fig = scatter2d(df.iloc[2:-3].index, slope_inside_coarse, inline=True)

In [ ]:
fig = scatter2d(df.index,slope_inside,inline=True);

In [ ]:
pio.write_image(fig, plotdir+"temp_change_Yoke.pdf")
pio.write_image(fig, plotdir+"temp_change_Yoke.png")

In [ ]:
fig = scatter2d(df.index,slope_inside2,inline=True);

In [ ]:
plot(fig)

In [ ]:
pio.write_image(fig, plotdir+"temp_change_Floor.pdf")
pio.write_image(fig, plotdir+"temp_change_Floor.png")

In [ ]:
slope_outside = [t_outside[i+1]-t_outside[i] for i in range(len(t_outside)-1)]
slope_outside.append(0)
slope_outside = pd.Series(slope_outside,name="Outside Temp Change")

In [ ]:
fig = scatter2d(df.index,slope_outside,inline=True);

In [ ]:
pio.write_image(fig, plotdir+"temp_change_Desk.pdf")
pio.write_image(fig, plotdir+"temp_change_Desk.png")

In [ ]:
fig = histo(slope_inside,bins=50,autobin=False,xlabel="dT/dt (deg / min)",inline=True);
pio.write_image(fig, plotdir+"hist_temp_change_Yoke.pdf")
pio.write_image(fig, plotdir+"hist_temp_change_Yoke.png")
fig = histo(slope_inside2,bins=50,autobin=False,xlabel="dT/dt (deg / min)",inline=True);
pio.write_image(fig, plotdir+"hist_temp_change_Floor.pdf")
pio.write_image(fig, plotdir+"hist_temp_change_Floor.png")
fig = histo(slope_outside,bins=50,autobin=False,xlabel="dT/dt (deg / min)",inline=True);
pio.write_image(fig, plotdir+"hist_temp_change_Desk.pdf")
pio.write_image(fig, plotdir+"hist_temp_change_Desk.png")

In [ ]:
df["NMR_smooth"] = df[" NMR [T]"].ewm(span = 10).mean()

In [ ]:
nmr = df[" NMR [T]"].values
nmr_smooth = df["NMR_smooth"].values

#slope_nmr = [nmr[i+1]-nmr[i] for i in range(len(nmr)-1)]
slope_nmr = [(nmr[i+1]-nmr[i]) / (df.index[i+1] - df.index[i]).seconds * 60 for i in range(len(nmr)-1)]
slope_nmr_smooth = [(nmr_smooth[i+1]-nmr_smooth[i]) / (df.index[i+1] - df.index[i]).seconds * 60 for i in range(len(nmr_smooth)-1)]


slope_nmr.append(0)
slope_nmr = pd.Series(slope_nmr,name="NMR Change")

slope_nmr_smooth.append(0)
slope_nmr_smooth = pd.Series(slope_nmr_smooth,name="NMR (smoothed) Change")

In [ ]:
fig = scatter2d(df.index, df[" NMR [T]"])

In [ ]:
#subset['Close'].ewm(span = 3600).mean()
fig = scatter2d(2*[df.index], [df[" NMR [T]"], df[" NMR [T]"].ewm(span = 30).mean()],inline=True)

In [ ]:
fig = scatter2d(df.index,slope_nmr,inline=True);
fig = scatter2d(df.index,slope_nmr_smooth,inline=True);

In [ ]:
fig = histo(slope_nmr,xlabel="dB/dt (T / min.)", bins=50,yscale='log',autobin=False,inline=True);
fig = histo(slope_nmr_smooth,xlabel="Smoothed dB/dt (T / min.)", bins=50,yscale='log',autobin=False,inline=True);

In [ ]:
fig = histo(slope_nmr,xlabel="dB/dt (T / min.)", bins=50,yscale='log',autobin=False,inline=True);

In [ ]:
pio.write_image(fig, plotdir+"nmr_change.pdf")
pio.write_image(fig, plotdir+"nmr_change.png")

In [ ]:
slope_nmr.autocorr()

In [ ]:
pd.plotting.autocorrelation_plottslope_nmrrrelation_plot(slope_nmr)

In [ ]:
pd.plotting.autocorrelation_plot(df[" NMR [T]"])

In [ ]:
#pd.TimedeltaIndex(df.index[100],df.index[99])
(df.index[100] - df.index[99]).seconds

In [ ]:
td.seconds

## Correlation Desk Ambient x 2

In [ ]:
sorted(df.columns)

In [ ]:
corr_main = "Desk Ambient"
corr_y = " Spare Ambient on desk"

#corr_main = ' Chamber Wall CH14'
#corr_y = ' Chamber Wall CH15'

#corr_main = ' Yoke'
#corr_y = ' NMR [T]'


filestart = corr_main.strip().lower()

#slope, intercept, r_value, p_value, std_err = stats.linregress(df[" Yoke"], df[" NMR [T]"])
slope, intercept, r_value, p_value, std_err = stats.linregress(df[corr_main], df[corr_y])

slope, intercept, r_value, p_value, std_err

In [ ]:
line = slope * df[corr_main] + intercept

fig = scatter2d([df[corr_main],df[corr_main]],[df[corr_y],line],
                lines=False,markers=True,size_list=[2,2],show_plot=False,inline=True)

In [ ]:
fig.update_layout(
    showlegend=False,
    annotations=[
        go.layout.Annotation(
            x=(df[corr_main].max()+df[corr_main].min())/2,
            y=(df[corr_y].max()+df[corr_y].min())/2,
            #xref="x",
            #yref="y",
            text=f"y = {slope:.3f} * x + {intercept:.3f}",
            #text=f"y = {slope:.3E} * x + {intercept:.3f}",
            #showarrow=False,
            font=dict(
                family="Courier New, monospace",
                size=16,
                color="#ffffff"
            ),
            align="center",
            #arrowhead=2,
            #arrowsize=1,
            #arrowwidth=2,
            #arrowcolor="#636363",
            ax=-200,
            ay=-100,
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor="#483366",
            opacity=0.8
        )
    ]
);


In [ ]:
pio.write_image(fig,plotdir+f"{filestart}_vs_{corr_y.strip().lower()}_fit.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_{corr_y.strip().lower()}_fit.png")

In [ ]:
df["NMR_const"] = df[" NMR [T]"] - slope * df[corr_main] - intercept + df[" NMR [T]"].mean()

fig = scatter2d(2*[df[corr_main]],[df["NMR_const"],line],lines=False,markers=True,size_list=[2,2],inline=True)

pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.pdf")
pio.write_image(fig,plotdir+f"{filestart}_vs_nmr_fit_subtract.png")

fig = histo(df[" NMR [T]"],bins=50,autobin=False,inline=True);
pio.write_image(fig,plotdir+f"hist_NMR.pdf")
pio.write_image(fig,plotdir+f"hist_NMR.png")
fig = histo(df["NMR_const"],bins=50,autobin=False,inline=True);
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.pdf")
pio.write_image(fig,plotdir+f"{filestart}_correction_nmr_hist.png")

In [ ]:
fig = scatter2d(df_full[::10].index, df_full[" Parameter HVAC sensor"][::10], inline=True)

In [ ]:
df[" Magnet Current [A]"].describe()

In [ ]:
df[" NMR [T]"].describe()

In [ ]:
fig = histo(df[" Magnet Current [A]"],xlabel="Current (A)", bins=50, autobin=False, inline=True)

In [ ]:
filename

In [ ]:
df_full.index[-1]

In [ ]:
df.index[-1]

## Smoothing + Change in Time
- Sandor pointed out in FMS #58 that a noisy signal is not good for doing point by point derivative estimate
- Instead smooth the data and then do the calculation.

In [ ]:
col_float = [col for col in df.columns if df[col].dtype == np.float64]

In [ ]:
'''temps_ch = [a for a in df.columns if ("CH" in a) and int(a[-2:])<21]# or "HVAC" in a]
temps_ch.append(" Hall Element")
temps_ch.append(" Floor")
temps_ch.append(" Roof")
temps_ch.append(" Yoke")
temps_ch.append("Yoke (center magnet)")'''

In [ ]:
# Example of smoothing
N = 60
col =  " Hall Element" # " Chamber Wall CH13" # " NMR [T]"
fig = scatter2d(2*[df.index], [df[col], df[col].rolling(window = N).mean()],inline=True)
fig = scatter2d([df.index], [df[col] - df[col].rolling(window = N).mean()],inline=True)
fig = histo(df[col] - df[col].rolling(window = N).mean(),xlabel=f"d{col} / dt (1 / min.)", bins=50,yscale='linear',autobin=False,show_plot=True,inline=True);

In [ ]:
N =  15 # 30 # 15 # window size

for col in [" Hall Element"]:#col_float:
    notsmooth = df[col]
    smooth = df[col].rolling(window = N).mean()
    
    slope = [(smooth[i+1]-smooth[i]) / (df.index[i+1] - df.index[i]).seconds * 60 for i in range(len(smooth)-1)]
    slope.append(0)
    slope = pd.Series(slope,name=f"{col} Change <br> (smoothed, window = {N} min.)")
    
    slope2 = [(notsmooth[i+1]-notsmooth[i]) / (df.index[i+1] - df.index[i]).seconds * 60 for i in range(len(notsmooth)-1)]
    slope2.append(0)
    slope2 = pd.Series(slope2,name=f"{col} Change <br> (not smoothed)")
    
    fig = scatter2d(df.index, slope, show_plot=False, inline=True)
    pio.write_image(fig, plotdir+f"smoothed/time/{col}_change_smooth.pdf")
    pio.write_image(fig, plotdir+f"smoothed/time/{col}_change_smooth.png")
    fig = histo(slope,xlabel=f"d{col} / dt (1 / min.)", bins=50,yscale='log',autobin=False,show_plot=True,inline=True);
    pio.write_image(fig, plotdir+f"smoothed/hist/hist_{col}_change_smooth.pdf")
    pio.write_image(fig, plotdir+f"smoothed/hist/hist_{col}_change_smooth.png")
    fig = histo(slope2,xlabel=f"d{col} / dt (1 / min.)", bins=50,yscale='log',autobin=False,show_plot=True,inline=True);

In [ ]:
fig = scatter2d()